In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
TRAINING_DIR = 'F:/FER2013_IMG/train'
TEST_DIR='F:/FER2013_IMG/test'
VAL_DIR='F:/FER2013_IMG/val'

In [3]:
num_classes = 7
IMG_SIZE = 48
BATCH_SIZE = 64
input_shape = (48,48,1)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
) 


Found 28709 images belonging to 7 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

Found 3589 images belonging to 7 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='categorical',
    
)

Found 3589 images belonging to 7 classes.


In [8]:
from tensorflow.keras.optimizers import Adam
import keras
                
base_model = keras.applications.VGG16(
             weights = 'imagenet',
             input_shape=(48, 48, 3),
             include_top=False)
                
base_model.trainable = False
                
x = base_model.output

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.6)(x)
                
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

                    
model = keras.Model(base_model.input, outputs)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.001),
                metrics=['accuracy',
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall')]
                        )
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [10]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
448/448 [==============================] - 620s 1s/step - loss: 1.6203 - accuracy: 0.3644 - precision: 0.5884 - recall: 0.0773 - val_loss: 1.5804 - val_accuracy: 0.3873 - val_precision: 0.6680 - val_recall: 0.0938
Epoch 2/5
448/448 [==============================] - 176s 394ms/step - loss: 1.6094 - accuracy: 0.3698 - precision: 0.5982 - recall: 0.0802 - val_loss: 1.5805 - val_accuracy: 0.3870 - val_precision: 0.6050 - val_recall: 0.1214
Epoch 3/5
448/448 [==============================] - 205s 457ms/step - loss: 1.6124 - accuracy: 0.3731 - precision: 0.5921 - recall: 0.0805 - val_loss: 1.5832 - val_accuracy: 0.3842 - val_precision: 0.6526 - val_recall: 0.0776
Epoch 4/5
448/448 [==============================] - 181s 403ms/step - loss: 1.6044 - accuracy: 0.3748 - precision: 0.5884 - recall: 0.0822 - val_loss: 1.5891 - val_accuracy: 0.3803 - val_precision: 0.6695 - val_recall: 0.0656
Epoch 5/5
448/448 [==============================] - 179s 400ms/step - loss: 1.6070 - accuracy:

In [11]:
model.save('F:/Xray_Gaus/Vgg16_FER_wt')

history.history

INFO:tensorflow:Assets written to: F:/Xray_Gaus/Vgg16_FER_wt\assets


{'loss': [1.6203203201293945,
  1.6094375848770142,
  1.6123743057250977,
  1.604427695274353,
  1.6070445775985718],
 'accuracy': [0.3644266128540039,
  0.3698027431964874,
  0.37308430671691895,
  0.37482979893684387,
  0.37311920523643494],
 'precision': [0.5883603692054749,
  0.5981770753860474,
  0.5921458005905151,
  0.5884115695953369,
  0.6017941832542419],
 'recall': [0.07729097455739975,
  0.08018851280212402,
  0.08053761720657349,
  0.08224821090698242,
  0.08430790901184082],
 'val_loss': [1.5803688764572144,
  1.5804712772369385,
  1.583208441734314,
  1.5890893936157227,
  1.5788218975067139],
 'val_accuracy': [0.3872767984867096,
  0.3869977593421936,
  0.3842075765132904,
  0.3803013265132904,
  0.3836495578289032],
 'val_precision': [0.6679920554161072,
  0.6050069332122803,
  0.6525821685791016,
  0.6695156693458557,
  0.6401446461677551],
 'val_recall': [0.09375,
  0.1213727667927742,
  0.0775669664144516,
  0.0655691996216774,
  0.0987723246216774]}

In [12]:
results = model.evaluate(test_gen, batch_size=128)

113/113 [==============================] - 36s 318ms/step - loss: 1.5396 - accuracy: 0.3993 - precision: 0.6335 - recall: 0.1137


In [13]:
y_pred = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)

113/113 [==============================] - 20s 176ms/step


In [14]:
y_true=test_gen.labels

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.15      0.04      0.06       491
           1       0.00      0.00      0.00        55
           2       0.13      0.08      0.10       528
           3       0.24      0.43      0.31       879
           4       0.14      0.08      0.10       594
           5       0.12      0.13      0.12       416
           6       0.18      0.23      0.20       626

    accuracy                           0.19      3589
   macro avg       0.14      0.14      0.13      3589
weighted avg       0.17      0.19      0.17      3589

C:\Users\mprad\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 18,   0,  52, 207,  45,  58, 111],
       [  1,   0,   6,  25,   6,   7,  10],
       [ 16,   0,  44, 224,  55,  63, 126],
       [ 35,   0,  79, 374,  79, 101, 211],
       [ 20,   0,  61, 267,  49,  74, 123],
       [ 16,   0,  43, 175,  48,  53,  81],
       [ 15,   0,  57, 273,  61,  77, 143]], dtype=int64)